<a href="https://colab.research.google.com/github/bjungweapon/mjc.ai.ml/blob/BDU/BUD_logisticregression_solver_animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

# 1. 데이터 생성
X, y = make_classification(n_samples=300, n_features=2,
                           n_redundant=0, n_informative=2,
                           n_clusters_per_class=1, random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 2. 초기화
solvers = ['sag', 'saga', 'sgd']
colors = ['r', 'g', 'b']
classifiers = [SGDClassifier(loss='log_loss', learning_rate='constant', eta0=0.01,
                             max_iter=1, warm_start=True, tol=None, random_state=42)
               for _ in solvers]

decision_lines = []

# 3. 시각화 준비
fig, ax = plt.subplots(figsize=(8, 6))
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx = np.linspace(x_min, x_max, 100)

# 산점도
ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1, edgecolor='k')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_title('Decision Boundaries: sag vs saga vs sgd')
lines = [ax.plot([], [], label=solver, color=color)[0] for solver, color in zip(solvers, colors)]
ax.legend()

# 4. 애니메이션 함수
def init():
    for line in lines:
        line.set_data([], [])
    return lines

def update(i):
    for idx, clf in enumerate(classifiers):
        clf.fit(X, y)
        coef = clf.coef_[0]
        intercept = clf.intercept_
        if coef[1] != 0:
            yy = -(coef[0] * xx + intercept) / coef[1]
            lines[idx].set_data(xx, yy)
    return lines

ani = FuncAnimation(fig, update, frames=30, init_func=init, blit=True, interval=200)

from IPython.display import HTML
HTML(ani.to_jshtml())


 주의사항
SGDClassifier는 sag, saga, sgd를 직접 선택하는 것이 아니라, 내부적으로 구현된 방식은 모두 SGD 기반이며, 우리는 애니메이션 목적상 sag, saga, sgd라는 라벨만 붙였습니다.

실제 LogisticRegression(solver='sag')은 내부가 조금 다릅니다.
하지만 시각적으로 SGD 기반 최적화 방식의 수렴 차이를 비교하는 데에는 적절합니다.

💡 이해 포인트
SGD는 noisy하고 진동이 많음

SAGA는 sparse 데이터나 L1/L2 regularization에 적합하고 빠른 수렴

SAG는 평균 gradient를 누적하며 안정적으로 수렴



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# 1. 데이터 생성 (적은 샘플)
np.random.seed(0)
n_samples = 50
X = np.random.randn(n_samples, 2)
true_w = np.array([2.0, -3.0])
y_prob = 1 / (1 + np.exp(-X @ true_w))
y = (y_prob > 0.5).astype(int)

# 2. sigmoid, gradient
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# 초기화
epochs = 10
lr = 0.1
w_sgd = np.zeros(2)
w_sag = np.zeros(2)
w_saga = np.zeros(2)

sag_memory = np.zeros((n_samples, 2))
saga_memory = np.zeros((n_samples, 2))

w_hist_sgd, w_hist_sag, w_hist_saga = [], [], []

for epoch in range(epochs):
    for i in range(n_samples):
        xi, yi = X[i], y[i]
        pred = sigmoid(xi @ w_sgd)
        grad = (pred - yi) * xi

        # SGD
        w_sgd -= lr * grad
        w_hist_sgd.append(w_sgd.copy())

        # SAG
        sag_memory[i] = grad
        w_sag -= lr * np.mean(sag_memory, axis=0)
        w_hist_sag.append(w_sag.copy())

        # SAGA
        old_grad = saga_memory[i].copy()
        saga_memory[i] = grad
        avg_grad = np.mean(saga_memory, axis=0)
        corrected_grad = grad - old_grad + avg_grad
        w_saga -= lr * corrected_grad
        w_hist_saga.append(w_saga.copy())

# 3. 애니메이션
w_hist_sgd = np.array(w_hist_sgd)
w_hist_sag = np.array(w_hist_sag)
w_hist_saga = np.array(w_hist_saga)

fig, ax = plt.subplots()
x_vals = np.linspace(-3, 3, 100)
line_sgd, = ax.plot([], [], 'r-', label='SGD')
line_sag, = ax.plot([], [], 'g--', label='SAG')
line_saga, = ax.plot([], [], 'b-.', label='SAGA')

def init():
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.legend()
    for xi, yi in zip(X, y):
        ax.plot(xi[0], xi[1], 'ko' if yi == 0 else 'ro', markersize=3)
    return line_sgd, line_sag, line_saga

def update(i):
    for line, w in zip([line_sgd, line_sag, line_saga],
                       [w_hist_sgd[i], w_hist_sag[i], w_hist_saga[i]]):
        if w[1] != 0:
            y_vals = -(w[0] * x_vals) / w[1]
            line.set_data(x_vals, y_vals)
    return line_sgd, line_sag, line_saga

ani = FuncAnimation(fig, update, frames=len(w_hist_sgd), init_func=init, blit=True)
HTML(ani.to_jshtml())
